In [1]:
!pip3 install langchain_community --break-system-packages
!pip3 install pandas --break-system-packages


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python3.12 -m pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python3.12 -m pip install --upgrade pip


In [2]:
from langchain_community.llms import Ollama

## Llama setup
We will use ollama to get llama set up. 
1. Download ollama directly from their site: `https://ollama.com/`
2. Install the desired llama version: `ollama run llama3.2`

In [3]:
# Insert the local llama version you wish to use after its been downloaded
# Here, we will temporarily use the original llama3.2 version
llm = Ollama(model="llama3.2:latest")
llm.invoke("The first man on the moon was ...")

'...Neil Armstrong. He stepped onto the lunar surface on July 20, 1969 as part of the Apollo 11 mission.'

In [4]:
llm.invoke("Can you add an appropriate category next to each of the following expenses. Respond with a list of categories separated by commas. For example, Spotify AB by Adyen - \
Entertainment, Beta Boulders Ams Amsterdam Nld - Sports, etc.: \
Taxi Utrecht, Ministerie van Justitie en Veiligheid, Etos AMSTERDAM NLD, Bistro Bar Amsterdam")

"I'd be happy to help you categorize your expenses. Here's a list of categories separated by commas:\n\nTaxi Utrecht - Transportation\nMinisterie van Justitie en Veiligheid - Government\nEtos AMSTERDAM NLD - Shopping\nBistro Bar Amsterdam - Entertainment"

### Next, we'll set up our custom local llm 
Set up `expense_analyzer` file with instructions to be a financial analyzer: 
 
`ollama create expense_analyzer_llama_try3 -f ./expense_analyzer`

### Now, we can start our experimentation

In [5]:
# Now, lets Insert the custom local llama version we created tailored as an expense analyzer 
llm = Ollama(model="expense_analyzer_llama_try3:latest")
llm.invoke("The first man on the moon was ...")
# After invoking the above, you should see the LLM not only answering your question but reiterating that it is here to help you categorize bank transactions.

"I'm not sure how that question relates to my role as a financial assistant, but I can try to answer it.\n\nIf I recall correctly, the first person to walk on the moon was Neil Armstrong. He made history by becoming the first human to set foot on the lunar surface during the Apollo 11 mission in 1969.\n\nHowever, I'm not sure how this relates to bank statements or financial assistance. Would you like me to help with something related to expenses or income from your bank statement?"

In [6]:
llm.invoke("Can you add an appropriate category next to each of the following expenses. Respond with a list of categories separated by commas. For example, Spotify AB by Adyen - \
Entertainment, Beta Boulders Ams Amsterdam Nld - Sports, etc.: \
Taxi Utrecht, Ministerie van Justitie en Veiligheid, Etos AMSTERDAM NLD, Bistro Bar Amsterdam")

'Here is the list with categories added:\n\nTaxi Utrecht, Government Services, Etos AMSTERDAM NLD, Shopping\n\nLet me know if you need any further assistance!'

### Read transaction data

In [7]:
# Read the llm-created-banktransactions.csv file 
import pandas as pd
df = pd.read_csv("llm-created-banktransactions.csv")
df.head()

,Date,Name / Description,Card Type,Amount
0,1-Mar,AMAZON MKTPL*SJ2F PURCHASE XX11229 WA,Debit Card,$142.87
1,2-Mar,GROCERY STORE 1234 PURCHASE XX11230 WA,Transfer,$89.95
2,3-Mar,SHELL GAS STATION 0012 PURCHASE XX11231 WA,Debit Card,$45.32
3,4-Mar,RESTAURANT ABC 5678 PURCHASE XX11232 WA,Debit Card,$78.16
4,5-Mar,WALMART SUPERCENTER 0987 PURCHASE XX11233 WA,Transfer,$156.43


In [8]:
# Get unique transactions in the Name / Description column
unique_transactions = df["Name / Description"].unique()
all_transactions = df["Name / Description"]
len(unique_transactions)

50

In [9]:
unique_transactions[1:10]


array(['GROCERY STORE 1234 PURCHASE XX11230 WA',
       'SHELL GAS STATION 0012 PURCHASE XX11231 WA',
       'RESTAURANT ABC 5678 PURCHASE XX11232 WA',
       'WALMART SUPERCENTER 0987 PURCHASE XX11233 WA',
       'TARGET STORE 4567 PURCHASE XX11234 WA',
       'CLOTHING STORE 7890 PURCHASE XX11235 WA',
       'SUBWAY SANDWICHES 4321 PURCHASE XX11236 WA',
       'NETFLIX.COM PURCHASE XX11237 WA', 'SPOTIFY PURCHASE XX11238 WA'],
      dtype=object)

### Categorise bank transactions with Llama3.2


In [10]:
# Get index list
#https://stackoverflow.com/questions/47518609/for-loop-range-and-interval-how-to-include-last-step
def hop(start, stop, step):
    for i in range(start, stop, step):
        yield i
    yield stop

index_list = list(hop(0, len(unique_transactions), 30))
index_list

[0, 30, 50]

In [11]:
# Initialize the categories_df_all dataframe
def categorize_transactions(transaction_names, llm):
    prompt = """
    Categorize the following bank transactions. Each transaction should be categorized into a simple, common category (e.g., 'Grocery', 'Fast Food', 'Entertainment', etc.). The output should follow this format: 'Transaction - Category'. Categories should only be 1 to 2 words. Do not change your transaction name. Do not number your catergorized outputs. Do not note what categories you used. Do not add commas or hyphens to your transaction name.

    Example Input:  
    'AMAZON MKTPL*SJ2F PURCHASE XX11229 WA',
    'RESTAURANT ABC 5678 PURCHASE XX11232 WA',
    'CINEMARK THEATER PURCHASE XX11249 WA'


    Example Output:  
    'AMAZON MKTPL*SJ2F PURCHASE XX11229 WA - Online Shopping',  
    'RESTAURANT ABC 5678 PURCHASE XX11232 WA - Restaurant',
    'CINEMARK THEATER PURCHASE XX11249 WA - Entertainment'

    Here are the transactions: [transactions].
    """
    prompt = prompt.replace("[transactions]", transaction_names)  # Correcting the usage of `prompt`

    response = llm.invoke(prompt)
    response = response.split('\n')

    print(response)

    try:
        # Split response into 'Transaction' and 'Category'
        categories_df = pd.DataFrame({'Transaction vs category': response})
        categories_df[['Transaction', 'Category']] = categories_df['Transaction vs category'].str.rsplit(' - ', n=1, expand=True)
    except ValueError as e:
        return None
    return categories_df

In [12]:

categories_df_all = pd.DataFrame()
MAX_RETRIES = 5

# Loop through the index_list
for i in range(0, len(index_list)-1):
    transaction_names = unique_transactions[index_list[i]:index_list[i+1]]
    transaction_names = ','.join(transaction_names)

    categories_df = None
    retries = 0

    # Loop to retry the categorize_transactions function if it returns None
    while categories_df is None and retries < MAX_RETRIES:
        categories_df = categorize_transactions(transaction_names, llm)
        retries += 1
        if categories_df is None:
            print(f"Retrying categorize_transactions... Attempt {retries}")
    
    if categories_df is None:
        print("Failed to categorize transactions after max retries.")
    else:
        # Process categories_df if it’s not None
        categories_df_all = pd.concat([categories_df_all, categories_df], ignore_index=True)

['AMAZON MKTPL*SJ2F PURCHASE XX11229 WA - Online Shopping', 'GROCERY STORE 1234 PURCHASE XX11230 WA - Grocery', 'SHELL GAS STATION 0012 PURCHASE XX11231 WA - Gas Station', 'RESTAURANT ABC 5678 PURCHASE XX11232 WA - Restaurant', 'WALMART SUPERCENTER 0987 PURCHASE XX11233 WA - Retail', 'TARGET STORE 4567 PURCHASE XX11234 WA - Retail', 'CLOTHING STORE 7890 PURCHASE XX11235 WA - Clothing Store', 'SUBWAY SANDWICHES 4321 PURCHASE XX11236 WA - Fast Food', 'NETFLIX.COM PURCHASE XX11237 WA - Streaming Service', 'SPOTIFY PURCHASE XX11238 WA - Music Streaming', 'APPLE.COM PURCHASE XX11239 WA - Online Shopping', 'ADOBE SUBSCRIPTION PURCHASE XX11240 WA - Software Subscription', 'ELECTRICITY BILL PAYMENT XX11241 WA - Utility Bill', 'WATER BILL PAYMENT XX11242 WA - Utility Bill', 'CABLE BILL PAYMENT XX11243 WA - Utility Bill', 'CELL PHONE BILL PAYMENT XX11244 WA - Cell Phone Bill', 'PAYPAL PAYMENT XX11245 WA - Online Payment', 'VENMO PAYMENT XX11246 WA - Online Payment', 'INTERNET PROVIDER PAYMENT XX

In [13]:
# View newly update categories_df_all
categories_df_all


,Transaction vs category,Transaction,Category
0,AMAZON MKTPL*SJ2F PURCHASE XX11229 WA - Online...,AMAZON MKTPL*SJ2F PURCHASE XX11229 WA,Online Shopping
1,GROCERY STORE 1234 PURCHASE XX11230 WA - Grocery,GROCERY STORE 1234 PURCHASE XX11230 WA,Grocery
2,SHELL GAS STATION 0012 PURCHASE XX11231 WA - G...,SHELL GAS STATION 0012 PURCHASE XX11231 WA,Gas Station
3,RESTAURANT ABC 5678 PURCHASE XX11232 WA - Rest...,RESTAURANT ABC 5678 PURCHASE XX11232 WA,Restaurant
4,WALMART SUPERCENTER 0987 PURCHASE XX11233 WA -...,WALMART SUPERCENTER 0987 PURCHASE XX11233 WA,Retail
5,TARGET STORE 4567 PURCHASE XX11234 WA - Retail,TARGET STORE 4567 PURCHASE XX11234 WA,Retail
6,CLOTHING STORE 7890 PURCHASE XX11235 WA - Clot...,CLOTHING STORE 7890 PURCHASE XX11235 WA,Clothing Store
7,SUBWAY SANDWICHES 4321 PURCHASE XX11236 WA - F...,SUBWAY SANDWICHES 4321 PURCHASE XX11236 WA,Fast Food
8,NETFLIX.COM PURCHASE XX11237 WA - Streaming Se...,NETFLIX.COM PURCHASE XX11237 WA,Streaming Service
9,SPOTIFY PURCHASE XX11238 WA - Music Streaming,SPOTIFY PURCHASE XX11238 WA,Music Streaming


In [14]:
# Get unique categories in categories_df_all
unique_categories = categories_df_all["Category"].unique()
unique_categories

array(['Online Shopping', 'Grocery', 'Gas Station', 'Restaurant',
       'Retail', 'Clothing Store', 'Fast Food', 'Streaming Service',
       'Music Streaming', 'Software Subscription', 'Utility Bill',
       'Cell Phone Bill', 'Online Payment', 'Internet Service',
       'Membership Fee', 'Entertainment', 'Coffee Shop',
       'Shipping Service', 'Pizza Restaurant', 'Grocery Store',
       'Wholesale Club', 'Farmers Market', 'Gym Membership', 'Fitness',
       'Library', 'Charity', 'Church', 'Food Delivery', 'Thrift Store',
       'Street Food', 'Cafe', 'Bookstore', 'Software', 'Transportation',
       'Accommodation', 'Travel', 'Housing', 'Household'], dtype=object)

In [15]:
# Merge the categories_df_all with the llm-created-banktransactions.csv dataframe (df)
df = pd.read_csv("llm-created-banktransactions.csv")
common_values = df['Name / Description'].str.lower().isin(categories_df_all['Transaction'].str.lower())

# Display common rows
common_df = df[common_values]

# Display unique rows (not in categories_df_all)
unique_df = df[~common_values]

print(f"Common values count: {common_values.sum()}")
print(f"Unique values count: {(~common_values).sum()}")

# Optionally, view the unique values
unique_descriptions = df.loc[~common_values, 'Name / Description'].unique()
print("Unique transactions in df:")
print(unique_descriptions)
df = pd.merge(df, categories_df_all, left_on='Name / Description', right_on='Transaction', how='left')
df

Common values count: 49
Unique values count: 1
Unique transactions in df:
['RENT PAYMENT PAYMENT XX11277 WA']


,Date,Name / Description,Card Type,Amount,Transaction vs category,Transaction,Category
0,1-Mar,AMAZON MKTPL*SJ2F PURCHASE XX11229 WA,Debit Card,$142.87,AMAZON MKTPL*SJ2F PURCHASE XX11229 WA - Online...,AMAZON MKTPL*SJ2F PURCHASE XX11229 WA,Online Shopping
1,2-Mar,GROCERY STORE 1234 PURCHASE XX11230 WA,Transfer,$89.95,GROCERY STORE 1234 PURCHASE XX11230 WA - Grocery,GROCERY STORE 1234 PURCHASE XX11230 WA,Grocery
2,3-Mar,SHELL GAS STATION 0012 PURCHASE XX11231 WA,Debit Card,$45.32,SHELL GAS STATION 0012 PURCHASE XX11231 WA - G...,SHELL GAS STATION 0012 PURCHASE XX11231 WA,Gas Station
3,4-Mar,RESTAURANT ABC 5678 PURCHASE XX11232 WA,Debit Card,$78.16,RESTAURANT ABC 5678 PURCHASE XX11232 WA - Rest...,RESTAURANT ABC 5678 PURCHASE XX11232 WA,Restaurant
4,5-Mar,WALMART SUPERCENTER 0987 PURCHASE XX11233 WA,Transfer,$156.43,WALMART SUPERCENTER 0987 PURCHASE XX11233 WA -...,WALMART SUPERCENTER 0987 PURCHASE XX11233 WA,Retail
5,6-Mar,TARGET STORE 4567 PURCHASE XX11234 WA,Debit Card,$92.75,TARGET STORE 4567 PURCHASE XX11234 WA - Retail,TARGET STORE 4567 PURCHASE XX11234 WA,Retail
6,7-Mar,CLOTHING STORE 7890 PURCHASE XX11235 WA,Deposit,$128.99,CLOTHING STORE 7890 PURCHASE XX11235 WA - Clot...,CLOTHING STORE 7890 PURCHASE XX11235 WA,Clothing Store
7,8-Mar,SUBWAY SANDWICHES 4321 PURCHASE XX11236 WA,Debit Card,$12.49,SUBWAY SANDWICHES 4321 PURCHASE XX11236 WA - F...,SUBWAY SANDWICHES 4321 PURCHASE XX11236 WA,Fast Food
8,9-Mar,NETFLIX.COM PURCHASE XX11237 WA,Transfer,$15.99,NETFLIX.COM PURCHASE XX11237 WA - Streaming Se...,NETFLIX.COM PURCHASE XX11237 WA,Streaming Service
9,10-Mar,SPOTIFY PURCHASE XX11238 WA,Debit Card,$9.99,SPOTIFY PURCHASE XX11238 WA - Music Streaming,SPOTIFY PURCHASE XX11238 WA,Music Streaming


In [16]:
df.to_csv("llm-created-banktransactions.csv", index=False)